In [5]:
import cv2
import numpy as np
import time

In [6]:
video=cv2.VideoCapture(0)
time.sleep(3)

#store the background in a loop
for i in range(60):
    ret,back=video.read() 
    back=np.flip(back,axis=1) #it was captured in flipped form..make it straight
    
    #we use loop bcoz to store background perfectly without noise
    
while(video.isOpened()):
    check,img=video.read()   #if check=false break
    if check==False:
        break
        
    img=np.flip(img,axis=1)
    hsv=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)  # hue=actual color,brightness=how much white or black is mixed,sat=amt of gray
    
    #hsv detects better than rgb  
    # detect and segment red color
    #lower range
    lower_red=np.array([0,120,50])
    upper_red=np.array([10,255,255])
    
    #hsv color range (lower)
    mask1=cv2.inRange(hsv,lower_red,upper_red)
    
    #upper range
    lower_red=np.array([170,120,70])
    upper_red=np.array([180,255,255])
    mask2=cv2.inRange(hsv,lower_red,upper_red)
    mask1=mask1+mask2
    
    #dilate the masked image ....dilate or morph the red color image with background
    mask1=cv2.morphologyEx(mask1,cv2.MORPH_OPEN,np.ones((3,3),np.uint8)) #opening the masked image
    mask1=cv2.morphologyEx(mask1,cv2.MORPH_DILATE,np.ones((3,3),np.uint8)) #dilate
    
    mask2=cv2.bitwise_not(mask1) #segmment out red color from entire frame
    res1=cv2.bitwise_and(img,img,mask=mask2) # red color is replaced with background..we have to add that to frame
    res2=cv2.bitwise_and(back,back,mask=mask1) #red is replaced with background
    
    final=cv2.addWeighted(res1,1,res2,1,0)
    cv2.imshow("final",final)
    key=cv2.waitKey(1)
    if key==ord('q'):
        break
        
video.release()
cv2.destroyAllWindows()
        